# Visualize a scattering event using my WebGL code

First, clone the GitHub repo: https://github.com/ageller/Scatter_WebGL

In [1]:
import os
import json
import pandas as pd
import numpy as np

from IPython.display import IFrame

import http.server
import socketserver

from threading import Thread

In [2]:
# define the directory where the code lives (this can be anywhere on your computer; just change the directory below)
directory = os.path.join("/home/ynad93/tsunami/cusp_data/testing_files")

## Create some new data to replace the default data packaged with repo

For this example, I will read in some data in a csv file that I created a while ago.  (Note that this data is downsampled, and in some parts of the time sequence the trajectories do not look smooth.)

The data will need to be reformatted into a dict and output as a json with the following structure.  Each primary key is a time, and for each time there are particles that have the word "Particle" in the key name.  Each particle has at least a key of "r" with [x,y,z] values (anything else is ignored).

Below is a visual example of how the data should be formatted:

```
{
"0.1": {
  "Particle1": {
    "r" : [-44.59797843122882, -1.602469437175624, -6.039267952136012e-14],
   },
  "Particle2": {
    "r" : [21.90496690915433, 1.108629663443225, -0.01596151404716814],
   },
  "Particle3": {
    "r" : [22.6930115220745, 0.4938397737323992, 0.01596151404722853],
   }
},
"0.2": { ...
```

In [34]:
# read in the data
times = np.load(os.path.join(directory, "t.npy"))/(2*np.pi)
posall = np.load(os.path.join(directory, "pos.npy"))
starIDs = np.arange(posall.shape[1])

In [35]:
np.round(times,6)

array([0.      , 0.      , 0.      , ..., 7.90351 , 7.903592, 7.903725])

In [48]:
# reformat and output to the json file
Nframes = times.size
i0 = 0
i1 = -1
if Nframes>1E4:
    N_2 = int(times.size/2)
    i0 = int(N_2+55000)
    i1 = int(N_2*2 -1)

newData = dict()
i = i0
for t in times[i0:i1]:
    newData[t] = {}
    j = 0
    for iden in starIDs:
        newData[t]['P' + str(iden)] = {'r': [posall[i,j,0], posall[i,j,1], posall[i,j,2]]}
        j+=1
    i+=1

# serialize json
json_object = json.dumps(newData, indent = 4)
 
# write to file (ScatterParts.json is the file name, and it lives in the code's data directory)
# Note: this will replace the default data set
with open(os.path.join(os.getcwd(),'data','ScatterParts.json'), "w") as outfile:
    outfile.write(json_object)

In [5]:
i0-i1

-200000

In [10]:
# define the port that you want (8000 is standard)
port = 8001

In [ ]:
# a simple class to start the http server in a thread (so that you still have access to the jupyter notebook)
class serverThread(Thread): 
    def run(self):
        handler = http.server.SimpleHTTPRequestHandler
        #os.chdir(directory)
        with socketserver.TCPServer(("", port), handler) as httpd:
            print("serving at port", port)
            httpd.serve_forever()

serverThread().start()

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/tmp/ipykernel_2598/252163901.py", line 6, in run


  File "/usr/lib/python3.8/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.8/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use


In [49]:
# create an iFrame to view the visualization in this notebook
IFrame("http://localhost:8001", width = 1200, height = 800)

127.0.0.1 - - [20/Jan/2023 16:37:37] "GET /data/ScatterParts.json HTTP/1.1" 200 -


In [ ]:
server